TeaLearning
==========

This notebook describes the process for implementing IBM's TeaLearning training method in Keras. It assumes some basic knowledge of keras (https://keras.io) as well as the TeaLearning method itself as outlined in ["Backpropogation for Energy-Efficient Neuromorphic Computing (Esser et al.)"](https://papers.nips.cc/paper/5862-backpropagation-for-energy-efficient-neuromorphic-computing.pdf) and ["Improving Classification Accuracy of Feedforward Neural Networks for Spiking Neuromorphic Chips (Yepes et al.)"](https://www.ijcai.org/proceedings/2017/0274.pdf). The version of TeaLearning implemented in this notebook is closer to that of the latter.

______________________________________________________________________________________________________________________

Before we do anything else, let's import the necessary libraries. We'll be using mostly Keras, with some help from TensorFlow and Numpy. We should also import the required python library packages.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import operator
import functools
import math

import tensorflow as tf
import numpy as np
from keras import backend as K
from keras import Model
#from keras.engine import Layer
from keras import initializers
from keras.models import Sequential
from keras.layers import Layer, Dropout, Flatten, Activation, Input, Lambda, concatenate
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.utils import to_categorical

To begin, we should look at the heart of what makes TeaLearning possible: the connections. Because of the constraints placed on weights in the TrueNorth architecture, TeaLearning focuses instead on learning connections. However, connections are binary values, and cannot be explicitly represented as floating point values. Yepes et al. circumvents this constraint by using both an *effective network* and a *shadow network*. The effective network is what actually gets deployed onto TrueNorth, with binary connections, and the the shadow network is what gets trained during backpropogation. Essentially, the shadow network represents connections as floating point values, and the effective network constrains them by setting connections to 1 if the shadow connections are greater than 0.5, and 0 otherwise. 

In our implementation, it may be easier to think of it this way: we have a (unconstrained) network in which we are training connections. Connections are stored and updated as floating point values. However, when we feed forward, we will constrain the connections using the constraint outlined above. 

As such, we will use the following TensorFlow functions to implement the constraints: `tf.round` and `tf.clip_by_value`. We will first round the connections so that they take on integer values, and then clip them between 0 and 1 so they are binary.

In order to implement this during backpropogation, we need to worry about the gradients of these functions. In particular, we would like for both of these functions to assumed to have a gradient of 1 so that they don't interfere with our calculations (regardless of their actual differentiability). This behavior is default in TensorFlow for the `clip_by_value` function, but if you try to use `tf.round` during training it will throw an error (because it does not have a defined gradient).

We can achieve our desired behavior using `tf.RegisterGradient`.

In [ ]:
@tf.RegisterGradient("CustomRound")
def _const_round_grad(unused_op, grad):
    return grad

This code tells TensorFlow how to handle the calculation for a custom gradient called "CustomRound". Whenever it comes across this gradient when creating its graph, it will simply return the current gradient entering the graph without altering it in any way.

Now that we've gotten that out of the way, let's begin defining our class.

In [ ]:
class Tea(Layer):
    def __init__(self,
                 units,
                 **kwargs):
        """Initializes a new TeaLayer.

        Arguments:
            units -- The number of neurons to use for this layer."""
        self.units = units
        # Needs to be set to `True` to use the `K.in_train_phase` function.
        self.uses_learning_phase = True
        super(Tea, self).__init__(**kwargs)

The `Tea` class inherits from the Keras `Layer` class. For more information about Keras layers and custom layers, see https://keras.io/layers/writing-your-own-keras-layers/. 

The initialization of the `Tea` layer can be fairly straightforawrd. The only parameter that really needs to be defined is the number of neurons in the layer (`units`.) A full version of a `Tea` layer has many more named parameters to enable customization of the training process, but we'll keep it simple for now.

The Keras Functional API allows us to abstract the fan-in and fan-out constraints of TrueNorth from the `Tea` layer. Each layer is equivalent to one `TrueNorth` core. Using lambda functions, we can handle data segmentation ourselves (we'll return to this later.)

Before we move on, take note of the following line: `self.uses_learning_phase = True`. We must set this to True so that the parent class knows that we have different behavior based on whether we are training or inferring. More on that later.

For now, let's implement the `build` function. This is where we initialize all of our variables. First, we need to create a helper function to initialize our weights. Because weights are not trained in TeaLearning, they are set to 1 and -1 so as to maximize information transfer. In a similar (but more simple) vein as Esser et al., we alternate between 1 and -1 for each axon. In Keras, this will look like a 2D tensor where each row represents an axon and each column represents a neuron. We will pass in a 2D shape where the first value is the number of axons and the second value is the number of neurons.

In [ ]:
def tea_weight_initializer(shape, dtype=np.float32):
    """Returns a tensor of alternating 1s and -1s, which is (kind of like)
    how IBM initializes their weight matrix in their TeaLearning
    literature.

    Arguments:
        shape -- The shape of the weights to intialize.

    Keyword Arguments:
        dtype -- The data type to use to initialize the weights.
                 (default: {np.float32})"""
    num_axons = shape[0]
    num_neurons = shape[1]
    ret_array = np.zeros((int(num_axons), int(num_neurons)), dtype=dtype)
    for axon_num, axon in enumerate(ret_array):
        if axon_num % 2 == 0:
            for i in range(len(axon)):
                ret_array[axon_num][i] = 1
        else:
            for i in range(len(axon)):
                ret_array[axon_num][i] = -1
    return tf.convert_to_tensor(ret_array)

Let's make sure that this funciton works. We'll run it on an example core with 10 axons and 5 neurons.

In [15]:
weight = tea_weight_initializer((10, 5))
tf.print(weight)

[[1 1 1 1 1]
 [-1 -1 -1 -1 -1]
 [1 1 1 1 1]
 ...
 [-1 -1 -1 -1 -1]
 [1 1 1 1 1]
 [-1 -1 -1 -1 -1]]


Looks like everything is in being initialized correctly! On TrueNorth, each row will be an axon, and each neuron will have the same weight array: `[1, -1, 1, -1]`. The first axon (corresponding to the first row) will have an index of 0. The second axon (corresponding to the second row) will have an index of 1. The third, an index of 0, and so on.

Now we can write the build function. This is pretty simple: we just need to add a variable for the weights (which are static, which is why we set `trainable=False`), the connections, and the biases.

In [16]:
def build(self, input_shape):
    assert len(input_shape) >= 2
    shape = (input_shape[-1], self.units)
    self.static_weights = self.add_weight(
        name='weights',
        shape=shape,
        initializer=tea_weight_initializer,
        trainable=False)
    # Intialize connections around 0.5 because they represent probabilities.
    self.connections = self.add_weight(
        name='connections',
        initializer=initializers.TruncatedNormal(mean=0.5),
        shape=shape)
    self.biases = self.add_weight(
        name='biases',
        initializer='zeros',
        shape=(self.units,))
    super(Tea, self).build(input_shape)

# Bind the method to our class
Tea.build = build

Now for the `call` function. This holds the feedforward code for the network.

Before we do that, let's make sure we understand what the input data is going to look like. Take MNIST as an example. We have 60,000 training images, each 
with 784 pixels, giving us a shape of (60000, 784). That being said, each core has a fan-in of 256, so for one `Tea` layer, the input will have a shape of (60000, 256).

However, we're going to add one complicaiton. In order to make up for loss of data from quantization, **spiking neural networks often encode information in multiple *ticks*.If an image is encoded in 4 ticks, then there are four chances for it to spike. A simple encoding scheme would have a pixel of value zero spike no times, a pixel of value 0.5 spike twice, and a pixel of value 1 spike four times.** Encoding is beyond the scope of this notebook, but we will still add the functionality to have multiple ticks to our network. We will place it in the second dimension, so that our new shape will be (60000, 1, 256). If we encoded using 4 ticks, our shape would be (60000, 4, 256). Now, all the spikes can be fed forward at once in training, and summed up at the end of the network.

Note: 
**- spike train matrix is different from snntorch format**


Now we're ready to tackle the feedforward code. So what does feeding forward look like? Let's try to visualize this. The first thing we're going to want to do is constrain our connections. Let's pretend each core only has 10 axons and 5 neurons again. This means that each partition should take 10 inputs and transform them into 5 outputs. This calls for a 10x5 matrix.

In [20]:
connections = tf.keras.backend.random_normal((10, 5), mean=0.5)
print(connections)

tf.Tensor(
[[-0.75974894 -0.9617777  -0.45150584  1.6838727   1.2933118 ]
 [ 0.17310095  0.34506187  0.70386386  2.524545    1.025511  ]
 [ 0.48173663  0.34254372  0.35053408 -0.2060557  -0.90162647]
 [ 1.0288098   0.59922755  1.590117   -0.12803072  0.48953682]
 [ 2.3429031   0.06145093  0.608148   -0.39494425  2.2360828 ]
 [-0.61321044  2.8879483   0.94183517  0.54221267  0.15978149]
 [ 0.49197307 -0.35340714 -0.83975816  0.72682905  0.4872419 ]
 [ 0.41817164 -0.4549545  -0.16032964  1.1766019  -0.92988086]
 [ 1.0362253   0.4476633  -1.0603403  -0.51316583  0.7337873 ]
 [ 0.61686486  2.168763   -0.2953332  -1.0902239   1.3313984 ]], shape=(10, 5), dtype=float32)


Now when we'll round and clip like we mentioned before.

In [21]:
connections = tf.round(connections)
connections = tf.clip_by_value(connections, 0, 1)
print(connections)

tf.Tensor(
[[0. 0. 0. 1. 1.]
 [0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0.]
 [1. 0. 1. 0. 1.]
 [0. 1. 1. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 1.]
 [1. 1. 0. 0. 1.]], shape=(10, 5), dtype=float32)


So what does this matrix mean in the context of TrueNorth? Let's look at an example matrix (since we're using RNGs yours will likely differ.)

```python
array([[0. 0. 0. 1. 1.]
       [0. 0. 1. 1. 1.]
       [0. 0. 0. 0. 0.]
       [1. 1. 1. 0. 0.]
       [1. 0. 1. 0. 1.]
       [0. 1. 1. 1. 0.]
       [0. 0. 0. 1. 0.]
       [0. 0. 0. 1. 0.]
       [1. 0. 0. 0. 1.]
       [1. 1. 0. 0. 1.]], dtype=float32)
```
       
If we look at the first row, we can tell that the first axon will be connected to the 1st neuron. Similarly, the second axon will be connected to the 1st neuron, 3rd neuron, and 5th neuron. 

From above, we also know that the first axon will have an index of 0, the second an index of 1, the third an index of 0, and so on. This means that the first will have a weight of 1 with all neurons, and the second a weight of -1, because we are giving all neurons a weight array of `[1, -1, 1, -1]`. This means that we can just multiply our connections by our weight matrix to get the effective weight matrix.

In [23]:
weights = tea_weight_initializer((10, 5))
effective_weights = tf.multiply(connections, weights)
print(effective_weights)

tf.Tensor(
[[ 0.  0.  0.  1.  1.]
 [-0. -0. -1. -1. -1.]
 [ 0.  0.  0.  0.  0.]
 [-1. -1. -1. -0. -0.]
 [ 1.  0.  1.  0.  1.]
 [-0. -1. -1. -1. -0.]
 [ 0.  0.  0.  1.  0.]
 [-0. -0. -0. -1. -0.]
 [ 1.  0.  0.  0.  1.]
 [-1. -1. -0. -0. -1.]], shape=(10, 5), dtype=float32)


Now, lets send in some input data. Because our input data will be normalized, we can take advantage of the round funciton to constrain our data to spikes.

In [25]:
x = tf.keras.backend.random_uniform((1, 10), minval=0.0, maxval=1.0)
print(x)

tf.Tensor(
[[0.9772897  0.84335697 0.5765927  0.8523091  0.46383393 0.2337116
  0.10032392 0.07879257 0.82871544 0.15498376]], shape=(1, 10), dtype=float32)


In [26]:
x = tf.round(x)
print(x)

tf.Tensor([[1. 1. 1. 1. 0. 0. 0. 0. 1. 0.]], shape=(1, 10), dtype=float32)


These are the axons which will receive spikes. If we had 

```python
tf.Tensor([[1. 1. 1. 1. 0. 0. 0. 0. 1. 0.]], shape=(1, 10), dtype=float32)
```

then we'd be recieving a spike on the 1st, 2nd, 4th, 6th, and 10th axons.

In [27]:
output = tf.matmul(x, effective_weights)
print(output)

tf.Tensor([[ 0. -1. -2.  0.  1.]], shape=(1, 5), dtype=float32)


The values in this array are the current potentials for our neurons at this point. If we had 
```python
tf.Tensor([[ 0. -1. -2.  0.  1.]], shape=(1, 5), dtype=float32)
```
then the 1st neuron would have a potential of -2, the second would have a potential of -1, the third a potential of -2, and so on. Now we need to apply the leak, which is equivalent to the biases in training. On TrueNorth, leak values have to be integers, so we'll round them when feeding forward, but store and update them as floating point values (like the connections.)

In [28]:
biases = tf.keras.backend.random_normal((1, 5))
print(biases)

tf.Tensor([[-0.4567531   0.46403372  0.15551522  0.09955545  0.43889487]], shape=(1, 5), dtype=float32)


In [29]:
biases = tf.round(biases)
print(biases)

tf.Tensor([[-0.  0.  0.  0.  0.]], shape=(1, 5), dtype=float32)


In [30]:
output = tf.add(output, biases)
print(output)

tf.Tensor([[ 0. -1. -2.  0.  1.]], shape=(1, 5), dtype=float32)


Those are our final neuron potentials. Now we need to see who spikes. For TeaLearning, neurons are set with a threshold of 0, a floor of 0, and a reset potential of 0. This means that neurons reset to 0 no matter what happens, i.e., they are memoryless. Additionally, they will spike if their potential is greater than or equal to zero. Our network is already memoryless, so all we need to do is add a greater than or equal to operation

In [31]:
# Cast the output because `tf.greater_equal` returns booleans
output = tf.cast(tf.greater_equal(output, 0.0), tf.float32)
print(output)

tf.Tensor([[1. 0. 0. 1. 1.]], shape=(1, 5), dtype=float32)


This is our final output. If we had

```python
tf.Tensor([[1. 0. 0. 1. 1.]], shape=(1, 5), dtype=float32)
```
that would be equivalent to the 1st and 2nd neurons spiking. If there were four partitions, the spikes from those 5 neurons would get concatenated with the spikes from the three other sets of 5 neurons to form a 1D array of 20 outputs. This is what would get convolved over in the next layer.

There's one last thing we have to worry about. During inference, it's fine to just use `tf.greater_equal`, but during training we need something that has a gradient. Following suit of Esser et al., we use the sigmoid function as this approximation. Remember in the initializer when we had the `self.uses_learning_phase = True` line? This is why we need it. During the training phase, we will feed forward using a sigmoid activaiton so that we can use backpropogation. During the test (or validaiton) phase, we will use the `tf.greater_equal` funciton be one-to-one with TrueNorth.

Our `call` code should now look pretty straightforward. We replace some `tf` funcitons with `K` functions, which is the Keras backend API. Since our backend is TensorFlow, it shouldn't really make a difference, but we might as well use Keras where we can for consistency. The `K.in_train_phase` function returns the first parameter if we are in train phase, and the second parameter if we are not. This is how we can implement the sigmoid/greater_equal functionality.

In [ ]:
def call(self, x):
    with tf.get_default_graph().gradient_override_map(
        {"Round":"CustomRound"}):
        # Constrain input
        x = tf.round(x)
        # Constrain connections
        connections = self.connections
        connections = tf.round(connections)
        connections = K.clip(connections, 0, 1)
        # Multiply connections with weights
        weighted_connections = connections * self.static_weights
        # Dot input with weighted connections
        output = K.dot(x, weighted_connections)
        # Constrain biases
        biases = tf.round(self.biases)
        output = K.bias_add(
            output,
            biases,
            data_format='channels_last'
        )
        # Apply activation / spike
        output = K.in_train_phase(
            K.sigmoid(output),
            tf.cast(tf.greater_equal(output, 0.0), tf.float32)
        )
    return output
    
# Bind the method to our class
Tea.call = call

Finally, we need to implement the `compute_output_shape` function. Keras uses this to help build the graph. We know that the number of neurons we have is the sum of the number of neurons in all the partitions, so we will replace the last dimesion of the input shape with that.

In [ ]:
def compute_output_shape(self, input_shape):
    assert input_shape and len(input_shape) >= 2
    assert input_shape[-1]
    output_shape = list(input_shape)
    output_shape[-1] = self.units
    return tuple(output_shape)
    
# Bind the method to our class
Tea.compute_output_shape = compute_output_shape

That's all there is to the `Tea` Layer. However, there's one complication. Because we have spiking neurons, it would be helpful to have a number of different neurons associated with each class, so that we can sum up all the guesses for each class and take the maximum. It's easy to imagine that only having one spiking neuron per class could make it very hard to make probabalistic guesses. So, we need to create a helper layer that can sum up all the neurons corresponding to a class. While we're at it, we can also up the guesses for each tick if we have multiple ticks.

The code for this is pretty straight forward, and the comments should be enough to clear up anything that is unclear.

In [ ]:
class AdditivePooling(Layer):
    """A helper layer designed to format data for output during TeaLearning.
    If the data input to the layer has multiple spikes per classification, the
    spikes for each tick are summed up. Then, all neurons that correspond to a
    certain class are summed up so that the output is the number of spikes for
    each class. Neurons are assumed to be arranged such that each
    `num_classes` neurons represent a guess for each of the classes. For
    example, if the guesses correspond to number from 0 to 9, the nuerons are
    arranged as such:

        neuron_num: 0  1  2  3  4  5  6  7  8  9  10 11 12  ...
        guess:      0  1  2  3  4  5  6  7  8  9  0  1  2   ..."""

    def __init__(self,
                 num_classes,
                 **kwargs):
        """Initializes a new `AdditivePooling` layer.

        Arguments:
            num_classes -- The number of classes to output.
        """
        self.num_classes = num_classes
        self.num_inputs = None
        super(AdditivePooling, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) >= 2
        # The number of neurons must be collapsable into the number of classes
        assert input_shape[-1] % self.num_classes == 0
        self.num_inputs = input_shape[-1]

    def call(self, x):
        # Sum up ticks if there are ticks
        if len(x.shape) >= 3:
            output = K.sum(x, axis=1)
        else:
            output = x
        # Reshape output
        output = tf.reshape(
            output,
            [-1, int(self.num_inputs / self.num_classes), self.num_classes]
        )
        # Sum up neurons
        output = tf.reduce_sum(output, 1)
        return output

    def compute_output_shape(self, input_shape):
        output_shape = list(input_shape)
        # Last dimension will be number of classes
        output_shape[-1] = self.num_classes
        # Ticks were summed, so delete tick dimension if exists
        if len(output_shape) >= 3:
            del output_shape[1]
        return tuple(output_shape)

We're finally ready to do some training! Let's start by loading the data

In [ ]:
# Load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# save old labels for later
y_test_not = y_test

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

Okay, now we'll define the model using the Keras Functional API. We'll use a Lambda layer to seperate our input into 4 different partitions of 256 pixels each so that each partition can be fed into one core. Then, we'll send them into Tea layers with 64 neurons each. Those 4\*64 outputs will be concatenated into one array of 256 values, which we'll send into an output core. The output core will have 250 neurons, which works out to be 25 neurons guessing per class (since we have 10 classes.) We'll send all of this into the AdditivePooling layer, which will output 10 values (one for each class) which we'll finally feed into a softmax function to aide training.

In [ ]:
# Define model (use functional API to follow fan-in and 
# fan-out constraints)
inputs = Input(shape=(28, 28,))
flattened_inputs = Flatten()(inputs)
# Send input into 4 different cores (256 axons each)
x0 = Lambda(lambda x : x[:,:256])(flattened_inputs)
x1 = Lambda(lambda x : x[:,176:432])(flattened_inputs)
x2 = Lambda(lambda x : x[:,352:608])(flattened_inputs)
x3 = Lambda(lambda x : x[:,528:])(flattened_inputs)
x0 = Tea(64)(x0)
x1 = Tea(64)(x1)
x2 = Tea(64)(x2)
x3 = Tea(64)(x3)
# Concatenate output of first layer to send into next
x = concatenate([x0, x1, x2, x3])
x = Tea(250)(x)
# Pool spikes and output neurons into 10 classes.
x = AdditivePooling(10)(x)
predictions = Activation('softmax')(x)

In [ ]:
model = Model(inputs=inputs, outputs=predictions)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          verbose=1,
          validation_split=0.2)

score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 2s 39us/step - loss: 0.5806 - acc: 0.8153 - val_loss: 0.4347 - val_acc: 0.8610
Epoch 2/5
48000/48000 [==============================] - 1s 29us/step - loss: 0.2902 - acc: 0.9107 - val_loss: 0.3639 - val_acc: 0.8840
Epoch 3/5
48000/48000 [==============================] - 1s 31us/step - loss: 0.2299 - acc: 0.9286 - val_loss: 0.2967 - val_acc: 0.9053
Epoch 4/5
48000/48000 [==============================] - 1s 29us/step - loss: 0.2026 - acc: 0.9373 - val_loss: 0.2807 - val_acc: 0.9116
Epoch 5/5
48000/48000 [==============================] - 1s 29us/step - loss: 0.1820 - acc: 0.9426 - val_loss: 0.2706 - val_acc: 0.9133
Test loss: 0.2618567518532276
Test accuracy: 0.9143


Now that we have a working network, we can use the `truenorthutils` package to output this network for the TrueNorth simulator.

In [ ]:
from truenorthutils.simulator import simulator_conversion
from truenorthutils.tealearning import create_cores, create_packets, Packet

To output for the simulator, we need to extract the connections, and biases from the network. We can use the `model.get_weights()` function to do this. This function returns a list of all the variables of the model. For each layer, the first variable is the connections, the second is the biases, and the third is the weights. `create_cores` expects a 2D list, where each inner list is a conceptual layer, and each item in the list is a core.

In [ ]:
all_weights = model.get_weights()
connections = [
    [np.clip(np.round(all_weights[0]), 0, 1).astype(int),
     np.clip(np.round(all_weights[3]), 0, 1).astype(int),
     np.clip(np.round(all_weights[6]), 0, 1).astype(int),
     np.clip(np.round(all_weights[9]), 0, 1).astype(int)],
    [np.clip(np.round(all_weights[12]), 0, 1).astype(int)]
]
biases = [
    [np.round(all_weights[1]).astype(int), 
     np.round(all_weights[4]).astype(int),
     np.round(all_weights[7]).astype(int),
     np.round(all_weights[10]).astype(int)],
    [np.round(all_weights[13]).astype(int)]
]
weights = [[all_weights[2], all_weights[5], all_weights[8], all_weights[11]], [all_weights[14]]]

In [ ]:
cores = create_cores(connections, weights, biases)

Now, we'll iterate through the first 10 images and create a 2D list of packet objects to be sent in to the simulator.

In [ ]:
# Coordinates of input core
core_coordinates = [(0, 0), (1, 0), (2, 0), (3, 0)]
# Convert the first 10 images to packets 
packets = []
for image_num, image in enumerate(x_test[:10]):
    temp_packets = []
    # Flatten image
    image = image.reshape(784)
    for i in range(0, 4):
        for pixel_num, pixel in enumerate(image[176*i:176*i+256]):
            if pixel > 0.5:
                temp_packets.append(
                    Packet(
                        core_coordinates[i],
                        pixel_num,
                        0
                    )
                )
    packets.append(temp_packets)

Finally, this function creats the `input.txt` and `labels.txt` functions for the simulator.

In [ ]:
simulator_conversion(cores, packets=packets, labels=y_test_not)

Now we can run the simulator with these files using the following command (within the `SoftwareSimulator/python_wrapper` directory

`python driver.py PATH_TO_INPUT/input.txt PATH_TO_LABELS/labels.txt PATH_FOR_OUTPUT/output.txt -t 11 -r 1`